In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from database_io.db_handler import DB_handler
import pymc as pm
import numpy as np
import pandas as pd

import plotly.express as px

In [ ]:
db = DB_handler()

In [ ]:
games = db.games.get_all_games(0.1)

In [ ]:
# games["minutes_missed"] = (90 - games["minutes"]).clip(lower=0)
games["elo_diff"] = games["elo"] - games["opposition_elo"]
games["goal_diff"] = games["result"].apply(lambda x: int(x.split("-")[0]) - int(x.split("-")[1]))
games["goal_home"] = games["result"].apply(lambda x: int(x.split("-")[0]))
games["goal_away"] = games["result"].apply(lambda x: int(x.split("-")[1]))


In [ ]:
games

In [ ]:
# games["elo_diff"] = games["elo_diff"] / (elo_std:=max(games["elo_diff"].max(), abs(games["elo_diff"].min())))
# games["goal_diff"] = games["goal_diff"] / (goal_std:=max(games["goal_diff"].max(), abs(games["goal_diff"].min())))
games["minutes_missed"] = (games["game_minutes"] - games["minutes"]).clip(lower=0)

# games["minutes_missed"] = games["minutes_missed"] / (min_std:=games["minutes_missed"].max())

In [ ]:
games_all = games.copy()

In [ ]:
games = games.tail(30_000)

In [ ]:
from ngboost import NGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


X_train, X_test, Y_train, Y_test = train_test_split(games[["home", "elo_diff", "minutes_missed"]], games["goal_diff"].values, test_size=0.2)

ngb = NGBRegressor().fit(X_train, Y_train, X_test, Y_test)
Y_preds = ngb.predict(X_test)
Y_dists = ngb.pred_dist(X_test)

test_MSE = mean_squared_error(Y_preds, Y_test)
print('Test MSE', test_MSE)

# test Negative Log Likelihood
test_NLL = -Y_dists.logpdf(Y_test).mean()
print('Test NLL', test_NLL)

In [ ]:
dist = ngb.pred_dist([[0, 300, 10]])

In [ ]:
dist.params

In [ ]:
print(dist.ppf(0.25))
print(dist.ppf(0.75))

In [ ]:
import pickle
from pathlib import Path

file_path = '/home/morten/Develop/packing-report/metrics/mov_elo/ngb.pckl'

with open(file_path, "wb") as f:
    pickle.dump(ngb, f)
